In [ ]:
#Import necessary libaries here.
import httplib, urllib, base64, json
import csv

#Replace the subscription_key string value with your valid subscription key.
subscription_key = '23e538cdd116475c86486613e9fc54e6'

#Replace or verify the region.
uri_base = 'westcentralus.api.cognitive.microsoft.com'

headers = {
    #Request headers.
    'Content-Type': 'application/json',
    'Ocp-Apim-Subscription-Key': subscription_key,
}

params = urllib.urlencode({
    #Request parameters. All of them are optional.
    'visualFeatures': 'Description',
    'language': 'en',
})


#The following function was written based on the documentation from Microsoft Azure
#website, along with some edit at the body for the url in our database to fit in.
#Also we made some changes at how the results will be shown. We only care about the
#images' descriptions and tags, not their scores (the scores would not be helpful
#for building our word bank). Hence, the function would return a list of strings 
#containing only the descriptions and tags of the input image.
def microsoft_vision(url):
    body="{'url':'http://%s'}"%url #format the body.
    descrp=''
    try:
        #Execute the REST API call and get the response.
        conn = httplib.HTTPSConnection('westcentralus.api.cognitive.microsoft.com')
        conn.request("POST", "/vision/v1.0/analyze?%s" % params, body, headers)
        response = conn.getresponse()
        data = response.read()

        #'data' contains the JSON data. The following formats the JSON data for display.
        parsed = json.loads(data)
        a= parsed['description']['captions'][0]['text'] #get the descriptions.
        b= ','.join(parsed['description']['tags']) #get the tags.
        c= a+','+b #combine descriptions and tags.
        descrp += c
        
        conn.close()
    
    except Exception as e:
        descrp+='error'
        print e
        
    return descrp

#Variables to update.
urls=[]
results=[]
counter=0
ecount=0

#Run the API on images from the csv file.
with open('images_bigdataset_B_filtered_allout.csv','rb') as old:
    reader=csv.DictReader(old)
    for row in reader:
        counter+=1
        urls.append(row['image_urls_B'])
        r= microsoft_vision(row['image_urls_B'])
        results.append(r) #append the processed API results from the photos.
        if r=='error':
            ecount+=1
        print counter, 'images processed', ecount, 'errors'
        
    pass

        
#Create a new csv file containing the processed API results.        
with open('final_results_B_images.csv','wb') as new:
    writer=csv.writer(new)
    writer.writerow(['image_urls_B','processed_api_results'])
    for i in range(0,len(urls)):
        a=[urls[i],results[i]]
        writer.writerow(a)
